# Árboles de Decisiones
____________


In [ ]:
import math
import warnings

# Desactivar advertencias de FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

### Carga de datos

#### Importation de las librerias basicas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carga el data set en una variable
prestamos = pd.read_excel('../datos/prestamos_bancarios_alemanes_1994.xls')

#### Exproracion de los datos

In [ ]:
prestamos.head()

In [ ]:
# Obtiene información de las filas y las columnas
prestamos.info()
total_registro = prestamos.shape[0]
print(f'Total de registro en el dataset: {total_registro}')

In [ ]:
prestamos.describe()

In [ ]:
prestamos.isnull().sum()

### Seleccion de las variables con las que vamos a trabajar

In [ ]:
categorias_elegidas = ["Account Balance","Duration of Credit (month)", "Payment Status of Previous Credit", "Purpose", "Sex & Marital Status"]
prestamos = prestamos[['Creditability'] + categorias_elegidas].copy()

### Limpieza de datos
* Se eliminan las filas con valores __NULL__
* Eliminar los duplicados

In [ ]:
prestamos = prestamos.dropna()
prestamos.isnull().sum()

In [ ]:
# Elimina los datos duplicados
prestamos.drop_duplicates(inplace=True)
# Verificar las datos duplicados
cantidad_duplicados = prestamos.duplicated().sum()
print(f'Total de filas duplicadas: {cantidad_duplicados}')

In [ ]:
prestamos.info()
total_registros = prestamos.shape[0]
print(f'Total de registro después de la limpeza: {total_registros}')

### Matriz de correlacion

In [ ]:
# Se crea la matriz
matriz = prestamos.corr()

# Gráfico de la matriz
# tamaño de la figura
plt.figure(figsize=(8,6))

# Crea el mapa de calor
sns.heatmap(matriz,annot=True, cmap='coolwarm',fmt='.2f')

# Agregar Titulo y etiquetas
plt.title('Matriz de correlaciones')
plt.show()

## Cuantificar la variables continuas
1. Duration of credit month


In [ ]:
# Estadisticas descriptivas de las variables continuas

estadistica_duration = prestamos['Duration of Credit (month)'].describe()

print(estadistica_duration)

In [ ]:
import pandas as pd

def cuantificar_variable(dataframe, nombreVar, num_grupos=4):
    # Ordenar el DataFrame según la columna a cuantificar
    dataframe = dataframe.sort_values(by=nombreVar, ascending=True).reset_index(drop=True)
    n = len(dataframe)
    
    # Calcular los límites de los cuartiles
    cuartiles = [dataframe[nombreVar].quantile(i / num_grupos) for i in range(1, num_grupos)]
    
    for i in range(len(dataframe)):
        for j, limite in enumerate(cuartiles):
            if dataframe.at[i, nombreVar] <= limite:
                dataframe.at[i, nombreVar] = j
                break
        else:
            dataframe.at[i, nombreVar] = num_grupos - 1

    return dataframe

prestamos = cuantificar_variable(prestamos, 'Duration of Credit (month)',num_grupos=4)


#### Diagramas de cajas de las variables
+ Creditability
+ Account Balance  
+ Duration of Credit (month)  
+ Payment Status of Previous Credit
+ Purpose 
+ Sex & Marital Status 

In [ ]:
# Creamos la figura
plt.figure(figsize=(15,10))

# Iteramos sobres la categorias elegidas y creamos un diagrama para cada una
for i, variable in enumerate(categorias_elegidas,1):
    plt.subplot(2,3,i) # Crea una cuadricula de 2x3
    sns.boxplot(data=prestamos, y=variable, x='Creditability')
    plt.title(f'Diagrama de caja de {variable}')
# Ajustar los graficos
plt.tight_layout()
plt.show()

## Division de los datos de entrenamientos, prueba y validacion
* Conjunto de entrenamientos = 60%
* Conjunto de validacion = 20%
* Conjunto de test = 20%

In [ ]:
# Separa las características (X) y la variable objetivo (y)
X = prestamos[['Account Balance', 'Duration of Credit (month)', 'Payment Status of Previous Credit', 'Purpose', 'Sex & Marital Status']]
y = prestamos['Creditability']


In [ ]:
import numpy as np

np.random.seed(42)

# Define las proporciones para la division del los conjuntos
ratios = [0.6,0.2,0.2] # 60% Entrenamiento, 20% validacion, 20% testeo

#Calcular el tamaño de cada conjunto
total_samples = len(X)
sizes = [int(r * total_samples) for r in ratios]

# General indices aleatorios
random_indices = np.random.permutation(total_samples)

#Division de los conjuntos
X_train = X.iloc[random_indices[:sizes[0]]]
X_val = X.iloc[random_indices[sizes[0]: sizes[0] + sizes[1]]]
X_test = X.iloc[random_indices[sizes[0] + sizes[1]:]]

y_train = y.iloc[random_indices[:sizes[0]]]
y_val = y.iloc[random_indices[sizes[0]:sizes[0] + sizes[1]]]
y_test = y.iloc[random_indices[sizes[0] + sizes[1]:]]

print("Cantidad de datos en el conjunto de entrenamiento:", X_train.shape[0])
print("Cantidad de datos en el conjunto de validación:", X_val.shape[0])
print("Cantidad de datos en el conjunto de prueba:", X_test.shape[0])


# Creacion del modelo - ___ÁRBOL DE DECISION___

### Calcular al Entropía de Shannon para todo el conjunto de datos
 $$\large H(S) = -p(S_{i'})\log_2(p(S_{i'})) - p(No)\log_2(p(No))$$ 


In [ ]:
def entropia_shannon(data, target_atributo):
    entropia = 0
    total_records = len(data)
    
    unique_classes = data[target_atributo].unique()
    
    for unique_class in unique_classes:
        class_records = data[data[target_atributo] == unique_class]
        class_count = len(class_records)
        probabilidad = class_count / total_records
        entropia -= round(probabilidad * math.log2(probabilidad),4)
    return entropia

entropia = entropia_shannon(prestamos, 'Creditability')
print(f' Calculo de la entropia del todo el conjunto de datos: {entropia}')

### Calculo de ___Ganancia___

$$\large G(S,A) = H(S) - ∑_{v\in A} \frac{|S_v|} ⋅ H(S_v)$$

In [ ]:
def calculo_ganancia(data,target_atributo, atributo):
    # Calculamos la entropia inicial
    entropia_s = entropia_shannon(data,target_atributo)
    
    # Obtener los valores unico del atributo
    valores_unicos = data[atributo].unique()
    
    # Calculo de la suma ponderada de las entropias despues de la division
    suma_entropia = 0
    total_registro = len(data)
    
    for value in valores_unicos:
        subset = data[data[atributo] == value]
        subset_size = len(subset)
        entropia_subset = entropia_shannon(subset,target_atributo)
        suma_entropia += (subset_size / total_registro) * entropia_subset
        
    # Calculo de la ganancia
    informacion_ganancia = entropia_s - suma_entropia
    return informacion_ganancia

# Calculamos la ganancia para todos los atributos
for column in prestamos.columns[1:]: # Se excluye el atributo 'Creditability'
    informacion_ganancia = calculo_ganancia(prestamos,'Creditability',column)
    print(f'Ganancia de la informacion para la {column} : {informacion_ganancia}')

## Seleccion de la Mejor Atributo


In [ ]:
atributos = prestamos.columns[1:]

#Inicia las variables 
mejor_carateristica = None
mejor_ganancia = -1 # inicia en valor muy bajo

# Calcula la ganancia de informacion para cada atributo y encuentra al mejor
for atributo in atributos:
    ganancia = calculo_ganancia(prestamos, 'Creditability',atributo)
    print(f'Ganancia de informacion para {atributo}: {ganancia}')
    
    if ganancia > mejor_ganancia:
        mejor_ganancia = ganancia
        mejor_carateristica = atributo

print('\n')    
print(f'La mejor caracteristica para dividir es : {mejor_carateristica}  con una ganancia de {mejor_ganancia} ')

## 